In [1]:
#!/usr/bin/env python
# coding: utf-8

# Workflow
# 
# 1. read all csv.gz data by timestamp in file name, e.g. SSL_SDE-RG01a_202210090845_791308.csv -> 2021-10-09 08:45. currently read daily
# 2. parse data and then find date col and output the data in parquet format, partition by date col, in pipeline_output_dir
# 3. move processed data files to 'derived_dir'
# 4. Use sslweb_delete.py to delete files in derived dir daily

from smtutil import hiveutil, oracle_to_hive
from datetime import datetime, timedelta
import re, glob, shutil, os
from shutil import copyfile
from pathlib import Path
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.sql.functions import lag, lead, first, last, udf, lit, when, expr
import pyspark.sql.functions as F
import yaml
import argparse

# schema
schema_raw = StructType([
    StructField("MSISDN", StringType(), True),
    StructField("IP1", StringType(), True),
    StructField("TIME_START", StringType(), True),
    StructField("IP2", StringType(), True),
    StructField("BYTE_COUNT_UP", IntegerType(), True),
    StructField("BYTE_COUNT_DOWN", IntegerType(), True),
    StructField("SSL_CERT_DOMAIN", StringType(), False),
    StructField("TIME_END", StringType(), True)])

# get binary time string
def hourly_string(l):
    output = ''
    for i in range(0, 24):
        if int(i) in l:
            output += '1'
        else:
            output += '0'
    return output
# define pyspark udf
udf_hourly_string = F.udf(lambda x: hourly_string(x), StringType())
# binary string to integer
udf_b2int = F.udf(hiveutil.binary2int)

def move_file(input_dir, output_dir, filename_format):

    for f in Path(input_dir).glob(filename_format): # grabs all files
        f.rename(Path(output_dir).joinpath(f.name)) # moves to target folder.

    print(f'[Info] Moved files from {input_dir} to {output_dir} with file format {filename_format}')
    return

In [2]:
# get spark session 
sc, spark = hiveutil.get_sc_spark()

In [5]:
sdf_20230518 = spark.read.option("basePath", "/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/").parquet("/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/date_id=20230518")

In [4]:
sdf_20230519 = spark.read.option("basePath", "/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/").parquet("/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/date_id=20230519")

In [2]:
# from pyspark.sql.types import DateType, StringType, StructField, StructType

# sdf_post_paid_trx = spark.createDataFrame(
#         [
#             ["20230520", "09710153", "11111111"],
#             ["20230520", "09710153", "22222222"],
#             ["20230520", "09966686", "33333333"]
#         ],
#         schema=StructType(
#             [
#                 StructField("date_id", StringType()),
#                 StructField("CUST_NUM", StringType()),
#                 StructField("SUBR_NUM", StringType()),
#             ]
#         ),
#     )

# print(sdf_post_paid_trx.count())
# sdf_post_paid_trx.write.mode('append').partitionBy('date_id').parquet("/app/ws-simon/data/test_weblog_20230520/")

In [11]:
spark.stop()
sc.stop()

In [4]:
cfg = '/app/ws-simon/git_repo/weblog/ssl_to_parquet/pipeline/raw/config_bdmrdev01.yaml'
ts = 20230518

# load yaml
fhandle = open(cfg,'r')
cfg = yaml.safe_load(fhandle)

# add time stamp if needed
if ts != '':
    cfg['time_stamp'] = ts
    print(f'[Info] time stamp: {cfg["time_stamp"]}')

    print('[Info] Configs:\n')
    for k, v in cfg.items():
        print(f'{k} : {v}')
    print('\n')

[Info] time stamp: 20230518
[Info] Configs:

input_dir : /app/t_ssl/raw/
working_dir : /app/t_ssl/working/
pipeline_output_dir : /app/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/
derived_dir : /app/weblog_ssl_housekeep/
filename_format : *_{}*.csv.gz
byte_to_kb_ratio : 1024
time_stamp : 20230518




In [5]:
# get spark session 
sc, spark = hiveutil.get_sc_spark()

# get time stamp
if cfg['time_stamp'] == '':
    cfg['time_stamp'] = datetime.strftime(datetime.now() - timedelta(days=1), '%Y%m%d')

# get file name format
filename_format = cfg['filename_format'].format(cfg['time_stamp'])
# print(f'[Info] Data Timestamp: {time_stamp}')
print(f'[Info] Filename Format: {filename_format}')

[Info] Filename Format: *_20230518*.csv.gz


In [6]:
time_stamp=cfg['time_stamp']

input_dir=cfg['input_dir']
working_dir=cfg['working_dir']
pipeline_output_dir=cfg['pipeline_output_dir']
derived_dir=cfg['derived_dir']

byte_to_kb_ratio=cfg["byte_to_kb_ratio"]

In [7]:
### 1. -------------------------------------- Move file to working directory --------------------------------------
move_file(input_dir, working_dir, filename_format)

[Info] Moved files from /app/t_ssl/raw/ to /app/t_ssl/working/ with file format *_20230518*.csv.gz


In [8]:
### 2. -------------------------------------- Read data file --------------------------------------
print(f'[Info] Reading Data Files.....')
print(datetime.now())

# print(f'[Info] Read csv files of timestamp {time_stamp}')
# read csv
sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

# drop na domain rows
sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
# print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

# print(f'[Info] End of Reading Data Files')
# print(datetime.now())

[Info] Reading Data Files.....
2023-05-22 11:51:07.307428
[Info] df size: 2851293713, 8


In [9]:
### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

# create date id for filtering and pertition
sdf = sdf.withColumn('date_id', F.date_format(F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss'), "yyyyMMdd"))
sdf = sdf.filter(F.col('date_id') == time_stamp)
    
# create necessary columns for grouping
sdf = sdf.withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
         .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
         .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
         .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
         .withColumn('MINUTES', F.col('SECONDS')/60) \
         .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP')))

In [16]:
print(sdf.count())
sdf.limit(1).toPandas()

2246204599


MSISDN          IP1      TIME_START            IP2  BYTE_COUNT_UP  \
0  85293191334  10.56.22.46  20230518114658  13.225.103.84           2694   

   BYTE_COUNT_DOWN  SSL_CERT_DOMAIN        TIME_END   date_id  \
0             8742  TAGS.TIQCDN.COM  20230518114741  20230518   

      START_TIMESTAMP       END_TIMESTAMP  SECONDS   MINUTES  START_HOUR  
0 2023-05-18 11:46:58 2023-05-18 11:47:41       43  0.716667          11

In [10]:
# group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
sdf2 = sdf.groupBy('MSISDN', 'date_id', 'SSL_CERT_DOMAIN').agg(
                F.count("*").alias('TOTAL_HIT'), \
                F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                F.sum('SECONDS').alias('SECONDS'), \
                F.sum('MINUTES').alias('MINUTES')
            )

In [18]:
print(sdf2.count())
sdf2.limit(1).toPandas()

266127279


MSISDN   date_id               SSL_CERT_DOMAIN  TOTAL_HIT  \
0  85219100034721  20230518  GSP57-SSL-LOCUS.LS.APPLE.COM          2   

  START_HOUR_SET  BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS   MINUTES  
0            [4]         108237            17630       19  0.316667

In [11]:
# add last few columns
sdf3 = sdf2.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                     .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                     .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                     .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET')))

In [20]:
sdf3.limit(1).toPandas()

MSISDN   date_id               SSL_CERT_DOMAIN  TOTAL_HIT  \
0  85219100034721  20230518  GSP57-SSL-LOCUS.LS.APPLE.COM          2   

  START_HOUR_SET  BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS   MINUTES  \
0            [4]         108237            17630       19  0.316667   

      SUBR_NUM  BYTE_COUNT    KB_COUNT  KB_COUNT_UP  KB_COUNT_DOWN  \
0  19100034721      125867  122.916992   105.700195      17.216797   

                HOUR_BINARY  
0  000010000000000000000000

In [12]:
# reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
sdf4 = sdf3.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                 F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                 F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                 udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                 F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                 'date_id')

In [13]:
sdf4.count()

266127279

In [11]:
sdf4.limit(1).toPandas()

266127279


subr_num               ssl_cert_domain  byte_count  hour_binary  \
0  19100034721  GSP57-SSL-LOCUS.LS.APPLE.COM      125867       524288   

   total_hit   date_id  
0          2  20230518

In [14]:
pipeline_output_dir

'/app/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'

In [17]:
temp = spark.read.option("basePath", pipeline_output_dir).parquet(pipeline_output_dir + "/date_id=20230521") 

In [21]:
temp.limit(1).show()

+-----------+---------------+----------+-----------+---------+--------+
|   subr_num|ssl_cert_domain|byte_count|hour_binary|total_hit| date_id|
+-----------+---------------+----------+-----------+---------+--------+
|19100090411| HB.ARALEGO.COM|     67673|    1048616|        7|20230521|
+-----------+---------------+----------+-----------+---------+--------+



In [27]:
schema_ssl = StructType(
            [
                StructField("subr_num", LongType()),
                StructField("ssl_cert_domain", StringType()),
                StructField("byte_count", LongType()),
                StructField("hour_binary", IntegerType()),
                StructField("total_hit", LongType()),
                StructField("date_id", StringType()),
            ]
        )

In [28]:
# sdf4.write.mode('append').partitionBy('date_id').parquet("/app/ws-simon/data/test_weblog_20230520/")
sdf4.write.option("schema", schema_ssl).mode('append').partitionBy('date_id').parquet(pipeline_output_dir)
# sdf5.coalesce(10).write.mode('append').partitionBy('date_id').parquet(f'{pipeline_output_dir}')

Py4JJavaError: An error occurred while calling o300.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [2]:
def processer(
                spark,
                time_stamp,
                filename_format,
                input_dir,
                working_dir,
                pipeline_output_dir,
                derived_dir,
                byte_to_kb_ratio=1024,
            ):

    ### 1. -------------------------------------- Move file to working directory --------------------------------------
    move_file(input_dir, working_dir, filename_format)

    ### 2. -------------------------------------- Read data file --------------------------------------
    print(f'[Info] Reading Data Files.....')
    print(datetime.now())

    # print(f'[Info] Read csv files of timestamp {time_stamp}')
    # read csv
    sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
    print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

    # drop na domain rows
    sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
    # print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

    # print(f'[Info] End of Reading Data Files')
    # print(datetime.now())

    ### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

    # create necessary columns for grouping
    sdf = sdf.withColumn('ACCESS_DATE', F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss')) \
                         .withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
                         .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
                         .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
                         .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
                         .withColumn('MINUTES', F.col('SECONDS')/60) \
                         .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP')))

    # group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
    sdf = sdf.groupBy('MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN').agg(
                    F.count("*").alias('TOTAL_HIT'), \
                    F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                    F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                    F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                    F.sum('SECONDS').alias('SECONDS'), \
                    F.sum('MINUTES').alias('MINUTES')
                )

    # add last few columns
    sdf = sdf.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                         .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                         .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                         .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                         .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                         .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET'))) \
                         .withColumn('date_id', F.date_format(F.col('ACCESS_DATE'), "yyyyMMdd"))

    # reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
    sdf = sdf.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                     F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                     F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                     udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                     F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                     'date_id')
    
    # filter only today's data
    sdf = sdf.filter(F.col('date_id') == time_stamp)

    # lower case column names
#     for col in sdf.columns:
#         sdf = sdf.withColumnRenamed(col, col.lower())

    ### 4. -------------------------------------- Save to parquet --------------------------------------

    # print(f'[Info] Start Transforming Data......')
    # print(datetime.now())

    # save to parquet
    sdf.coalesce(10).write.mode('append').partitionBy('date_id').parquet(f'{pipeline_output_dir}')

    # print(f'[Info] Saved Data to {pipeline_output_dir}')
    # print(datetime.now())

    ### 5. -------------------------------------- Move Processed data to derived --------------------------------------

    # move processed files to derived folder
    move_file(working_dir, derived_dir, filename_format)

    # print(f"[Info] Moved Processed Data to {derived_dir}")
    print(f"[Info] Finished Processing Data with timestamp {time_stamp}")
    print(datetime.now())

    return

In [4]:
def main(
            filename_format,
            input_dir,
            working_dir,
            pipeline_output_dir,
            derived_dir,
            time_stamp='',
            byte_to_kb_ratio=1024,
        ):
    """
    time_stamp: timestamp to select data files to process
    filename_format: filename format, e.g. '*_{}*.csv.gz', used format to replace {} with timestamp
    input_dir: directory that contains the raw data from DW
    working_dir: directory that contains the processing data
    pipeline_output_dir: directory that contains the output parquet files
    derived_dir: directory that contains processed raw data from DW
    byte_to_kb_ratio: ratio for converting byte to kb
    """

    # get spark session 
    sc, spark = hiveutil.get_sc_spark()

    # get time stamp
    if time_stamp == '':
        time_stamp = datetime.strftime(datetime.now() - timedelta(days=1), '%Y%m%d')

    # get file name format
    filename_format = filename_format.format(time_stamp)
    # print(f'[Info] Data Timestamp: {time_stamp}')
    print(f'[Info] Filename Format: {filename_format}')

    # process data
    processer(
        spark=spark, 
        time_stamp=time_stamp,
        filename_format=filename_format,
        input_dir=input_dir,
        working_dir=working_dir,
        pipeline_output_dir=pipeline_output_dir,
        derived_dir=derived_dir
    )

    spark.stop()
    sc.stop()

    return

# if __name__ == '__main__':

#     # parser
#     parser = argparse.ArgumentParser(description='Self Defined Timestamp')
#     parser.add_argument('--ts', type=str, default='',
#                         help='timestamp of the date of data files')
#     parser.add_argument('--cfg', type=str, default='/app/sna_bdmrdev03/ws-simon/git_repo/weblog/ssl_to_parquet/testing/config_test.yaml',
#                         help='.yaml config file path')
#     args = parser.parse_args()

#     # load yaml
#     fhandle = open(args.cfg,'r')
#     cfg = yaml.safe_load(fhandle)

#     # add time stamp if needed
#     if args.ts != '':
#         cfg['time_stamp'] = args.ts
#         print(f'[Info] time stamp: {cfg["time_stamp"]}')

# #     print('[Info] Configs:\n')
# #     for k, v in cfg.items():
# #         print(f'{k} : {v}')
# #     print('\n')

#     main(**cfg)